In [5]:
import os
import sys
from dataclasses import dataclass
from pathlib import Path

In [2]:
os.getcwd()

'/Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans/notebooks'

In [3]:
os.chdir("../")
%pwd

'/Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans'

In [7]:
import tensorflow as tf
saved_model = tf.keras.models.load_model('models/prepare_base_model/base_model_updated.h5')


In [72]:
@dataclass
class EvaluationConfig:
    path_of_model: Path
    validation_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
    

In [73]:
MLFLOW_TRACKING_URI="https://dagshub.com/goldirana1111/kidney_scans.mlflow"
MLFLOW_TRACKING_USERNAME="goldirana1111"
MLFLOW_TRACKING_PASSWORD="3148113060766cb32b96f89005cd20a2d3ce06d9"


In [77]:
from src.common import read_yaml, save_json, config_path, params_path
class ConfigurationManager:
    def __init__(self):
        self.config_path = config_path
        self.params_path = params_path
        
        self.config = read_yaml(self.config_path)
        self.params = read_yaml(self.params_path)
        print(self.config)
        
    def get_evaluation_config(self) -> EvaluationConfig:
        return EvaluationConfig(
                path_of_model = self.config.prepare_base_model,
                validation_path = self.config.data_ingestion.validation_path,
                all_params = self.params,
                mlflow_uri = "https://dagshub.com/goldirana1111/kidney_scans.mlflow",
                params_image_size = self.params.IMAGE_SIZE,
                params_batch_size = self.params.BATCH_SIZE
        )

In [78]:
o = ConfigurationManager()
# print(o.get_evaluation_config())

{'data': 'data/raw', 'data_ingestion': {'root_dir': 'data/', 'source_url': 'https://drive.google.com/file/d/1ZF74F8h_419Lf-jK-k9TRMHY1GazwSQd/view?usp=drive_link', 'zip_path': 'data/raw/data.zip', 'extract_path': 'data/processed', 'validation_path': 'data/validation'}, 'prepare_base_model': {'root_dir': 'models/prepare_base_model', 'base_model_path': 'models/prepare_base_model/base_model.h5', 'updated_base_model_path': 'models/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'models/training', 'trained_model_path': 'models/training/model.h5'}}


In [79]:
o.get_evaluation_config()

EvaluationConfig(path_of_model=ConfigBox({'root_dir': 'models/prepare_base_model', 'base_model_path': 'models/prepare_base_model/base_model.h5', 'updated_base_model_path': 'models/prepare_base_model/base_model_updated.h5'}), validation_path='data/validation', all_params=ConfigBox({'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}), mlflow_uri='https://dagshub.com/goldirana1111/kidney_scans.mlflow', params_image_size=BoxList([224, 224, 3]), params_batch_size=16)

In [85]:
import mlflow.keras
from urllib.parse import urlparse


In [86]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1./255,
            validation_split=0.30
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_path,
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model.updated_base_model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
# validation_data

In [87]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

{'data': 'data/raw', 'data_ingestion': {'root_dir': 'data/', 'source_url': 'https://drive.google.com/file/d/1ZF74F8h_419Lf-jK-k9TRMHY1GazwSQd/view?usp=drive_link', 'zip_path': 'data/raw/data.zip', 'extract_path': 'data/processed', 'validation_path': 'data/validation'}, 'prepare_base_model': {'root_dir': 'models/prepare_base_model', 'base_model_path': 'models/prepare_base_model/base_model.h5', 'updated_base_model_path': 'models/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'models/training', 'trained_model_path': 'models/training/model.h5'}}
Found 13 images belonging to 2 classes.
1/1 [==============================] - 1s 1s/step - loss: 0.8086 - accuracy: 0.6923


2023/12/30 18:32:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/rt/_sv9t5px3yv5bth97z8py2mr0000gn/T/tmp81rnkbe7/model/data/model/assets


/Users/goldyrana/miniconda3/envs/kidney_scans/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
